In [ ]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API key: ")

In [ ]:
# LLM interface
from langchain_google_genai.llms import GoogleGenerativeAI

model = GoogleGenerativeAI(model="gemini-2.0-flash")
model.invoke("The grass is")

'The grass is **green**.'

In [ ]:
# Chat interface
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
prompt = [HumanMessage(content="What is the capital of Germany?")]
model.invoke(prompt)

AIMessage(content='The capital of Germany is **Berlin**.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--28602790-8db6-459d-9eee-40c0f842c4a0-0', usage_metadata={'input_tokens': 7, 'output_tokens': 9, 'total_tokens': 16, 'input_token_details': {'cache_read': 0}})

In [ ]:
# Roles: HumanMessage, AIMessage, SystemMessage, ChatMessage
system_msg = SystemMessage(
    "You are a helpful assistant who responds to questions using bold text."
)
human_msg = HumanMessage("What is the capital of Germany?")
prompt = [system_msg, human_msg]
model.invoke(prompt)

AIMessage(content='**The capital of Germany is Berlin.**', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--e08d2873-75d5-4094-91dd-e9e787831919-0', usage_metadata={'input_tokens': 20, 'output_tokens': 9, 'total_tokens': 29, 'input_token_details': {'cache_read': 0}})

In [ ]:
from langchain_core.prompts import PromptTemplate

template = PromptTemplate.from_template(
    """Summarize the following text in 1 sentence. 
If the text is too short or lacks meaningful content, respond with "Not enough information to summarize."

Text: {text}

Summary: """
)

# Example input
template.invoke(
    {
        "text": """LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications.
    As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis."""
    }
)

StringPromptValue(text='Summarize the following text in 1 sentence. \nIf the text is too short or lacks meaningful content, respond with "Not enough information to summarize."\n\nText: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications.\n    As a language model integration framework, LangChain\'s use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\nSummary: ')

In [ ]:
# Templates make prompts reusable
template = PromptTemplate.from_template(
    """Use the following context to answer 
    the question. If the answer is not available in the context, respond with 
    "Not enough information provided".

Context: {context}

Question: {question}

Answer: """
)

prompt = template.invoke(
    {
        "context": """Video games come in many genres. Action games focus on fast 
        movement and quick reflexes. Puzzle games require players to think and 
        solve problems. Role-playing games, or RPGs, let players explore worlds 
        and complete quests as different characters.""",
        "question": "What type of games involve solving problems?",
    }
)

prompt

StringPromptValue(text='Use the following context to answer \n    the question. If the answer is not available in the context, respond with \n    "Not enough information provided".\n\nContext: Video games come in many genres. Action games focus on fast \n        movement and quick reflexes. Puzzle games require players to think and \n        solve problems. Role-playing games, or RPGs, let players explore worlds \n        and complete quests as different characters.\n\nQuestion: What type of games involve solving problems?\n\nAnswer: ')

In [ ]:
model.invoke(prompt)

'Puzzle games'

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Answer the question based on the context below. If the 
        question cannot be answered using the information provided, answer with 
        "I don\'t know".""",
        ),
        ("human", "Context: {context}"),
        ("human", "Question: {question}"),
    ]
)

prompt = template.invoke(
    {
        "context": """Modern gaming has evolved rapidly with the introduction of powerful graphics engines and online multiplayer capabilities. Popular platforms include PC, PlayStation, Xbox, and Nintendo Switch. Gamers often use services like Steam, PlayStation Network, and Xbox Live to access a vast library of titles and connect with friends around the world.""",
        "question": "Which platforms are popular for gaming?",
    }
)

prompt

ChatPromptValue(messages=[SystemMessage(content='Answer the question based on the context below. If the \n        question cannot be answered using the information provided, answer with \n        "I don\'t know".', additional_kwargs={}, response_metadata={}), HumanMessage(content='Context: Modern gaming has evolved rapidly with the introduction of powerful graphics engines and online multiplayer capabilities. Popular platforms include PC, PlayStation, Xbox, and Nintendo Switch. Gamers often use services like Steam, PlayStation Network, and Xbox Live to access a vast library of titles and connect with friends around the world.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Question: Which platforms are popular for gaming?', additional_kwargs={}, response_metadata={})])

In [ ]:
model.invoke(prompt)

AIMessage(content='PC, PlayStation, Xbox, and Nintendo Switch.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--2fbbcc12-9bbe-488e-9a29-43d3a9ab7938-0', usage_metadata={'input_tokens': 104, 'output_tokens': 11, 'total_tokens': 115, 'input_token_details': {'cache_read': 0}})

In [ ]:
# JSON Output
from pydantic import BaseModel


class GamingAnswerWithJustification(BaseModel):
    """An answer to the user's gaming question along with justification for the
    answer."""

    answer: str
    """The answer to the user's gaming question"""
    justification: str
    """Justification for the answer"""


structured_llm = model.with_structured_output(GamingAnswerWithJustification)

result = structured_llm.invoke(
    """Which platform is more popular for online multiplayer gaming, PC or console?"""
)
result

GamingAnswerWithJustification(answer='It is difficult to definitively say which platform is more popular.', justification='Both PC and consoles have large and active online multiplayer communities. The preferred platform often comes down to personal preference, the specific games someone enjoys, and their budget. Generally, consoles are more straightforward to set up for multiplayer, while PCs offer more customization and potentially better graphics, but at a higher cost. Recent trends suggest a blurring of the lines, with cross-platform play becoming more common, allowing players on different platforms to play together. Therefore, popularity is subjective and varies depending on the games and communities being considered. ')

In [ ]:
result.model_dump_json()

'{"answer":"It is difficult to definitively say which platform is more popular.","justification":"Both PC and consoles have large and active online multiplayer communities. The preferred platform often comes down to personal preference, the specific games someone enjoys, and their budget. Generally, consoles are more straightforward to set up for multiplayer, while PCs offer more customization and potentially better graphics, but at a higher cost. Recent trends suggest a blurring of the lines, with cross-platform play becoming more common, allowing players on different platforms to play together. Therefore, popularity is subjective and varies depending on the games and communities being considered. "}'

In [ ]:
# Output parsing
from langchain_core.output_parsers import CommaSeparatedListOutputParser

parser = CommaSeparatedListOutputParser()
parser.invoke("apple, banana, cherry")

['apple', 'banana', 'cherry']